In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

c:\Users\jaesc2\GitHub\skforecast


In [10]:
import re
import pytest
import joblib
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.exceptions import NotFittedError
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import HistGradientBoostingRegressor
from lightgbm import LGBMRegressor

from skforecast.exceptions import IgnoredArgumentWarning
from skforecast.preprocessing import RollingFeatures
from skforecast.preprocessing import TimeSeriesDifferentiator
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.ForecasterAutoregMultiSeries import ForecasterAutoregMultiSeries

from skforecast.ForecasterAutoreg.tests.fixtures_ForecasterAutoreg import data  # to test results when using differentiation


In [49]:
from skforecast.ForecasterAutoregMultiSeries.tests.fixtures_ForecasterAutoregMultiSeries import series
from skforecast.ForecasterAutoregMultiSeries.tests.fixtures_ForecasterAutoregMultiSeries import exog
from skforecast.ForecasterAutoregMultiSeries.tests.fixtures_ForecasterAutoregMultiSeries import exog_predict

series_dict = joblib.load(r'C:\Users\jaesc2\GitHub\skforecast\skforecast\ForecasterAutoregMultiSeries\tests\fixture_sample_multi_series.joblib')
exog_dict = joblib.load(r'C:\Users\jaesc2\GitHub\skforecast\skforecast\ForecasterAutoregMultiSeries\tests\fixture_sample_multi_series_exog.joblib')
end_train = "2016-07-31 23:59:00"
series_dict_train = {k: v.loc[:end_train,] for k, v in series_dict.items()}
exog_dict_train = {k: v.loc[:end_train,] for k, v in exog_dict.items()}
series_dict_test = {k: v.loc[end_train:,] for k, v in series_dict.items()}
exog_dict_test = {k: v.loc[end_train:,] for k, v in exog_dict.items()}

series_2 = pd.DataFrame({'1': pd.Series(np.arange(start=0, stop=50, dtype=float)), 
                         '2': pd.Series(np.arange(start=50, stop=100, dtype=float))})

In [62]:
forecaster = ForecasterAutoregMultiSeries(
    regressor          = LGBMRegressor(
        n_estimators=2, random_state=123, verbose=-1, max_depth=2
    ),
    lags               = 5,
    encoding           = None,
    dropna_from_series = False,
    transformer_series = StandardScaler(),
    transformer_exog   = StandardScaler()
)
forecaster.fit(
    series=series_dict_train, exog=exog_dict_train, suppress_warnings=True
)
levels = ['id_1000', 'id_1001', 'id_1003', 'id_1004', 'id_1005']
last_window = pd.DataFrame(
    {k: v for k, v in forecaster.last_window_.items() if k in levels}
)
last_window['id_1005'] = last_window['id_1004']
exog_dict_test_2 = exog_dict_test.copy()
exog_dict_test_2['id_1005'] = exog_dict_test_2['id_1004']
results = forecaster.create_predict_X(
    steps=5, levels=levels, last_window=last_window, exog=exog_dict_test_2, suppress_warnings=True
)
results['id_1000'].to_numpy()

array([[-0.98327068, -0.88248605, -0.79842631, -0.79277912, -0.77768307,
         0.00821644,  1.42962482,         nan,         nan],
       [-0.12514786, -0.98327068, -0.88248605, -0.79842631, -0.79277912,
         1.11220226,  0.89634375,         nan,         nan],
       [-0.12514786, -0.12514786, -0.98327068, -0.88248605, -0.79842631,
         1.38486425, -0.30192795,         nan,         nan],
       [-0.07645311, -0.12514786, -0.12514786, -0.98327068, -0.88248605,
         0.62088235, -1.26286725,         nan,         nan],
       [-0.07645311, -0.07645311, -0.12514786, -0.12514786, -0.98327068,
        -0.60444947, -1.26286725,         nan,         nan]])

In [67]:
results['id_1005'].to_numpy()

array([[ 1.22899343,  1.65819658,  2.4318926 ,  2.56743042,  2.38367241,
         0.00821644,  1.42962482,  1.11141113, -0.87943526],
       [ 0.12663112,  1.22899343,  1.65819658,  2.4318926 ,  2.56743042,
         1.11220226,  0.89634375,  1.1327558 ,  0.0058948 ],
       [ 0.01290134,  0.12663112,  1.22899343,  1.65819658,  2.4318926 ,
         1.38486425, -0.30192795,  1.1775869 , -0.3532584 ],
       [ 0.01290134,  0.01290134,  0.12663112,  1.22899343,  1.65819658,
         0.62088235, -1.26286725,  1.0428337 ,  0.84287284],
       [ 0.01290134,  0.01290134,  0.01290134,  0.12663112,  1.22899343,
        -0.60444947, -1.26286725,  1.00599776, -0.62314633]])